<a href="https://colab.research.google.com/github/MaanasGantla/MaanasGantla/blob/main/SpuCo_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Task:**

Deep neural networks often exploit non-predictive features that are spuriously correlated with class labels, leading to poor performance on groups of examples without such features. Using the SpuCo Package (SpuCo Documentation), we'd like you to implement a simple method to remedy spurious correlations in SpuCoMNIST (use default parameters to initialize the dataset).

The method (George) we'd like you to implement has a 3 step pipeline:
1. Train a model using ERM
2. Cluster inputs based on the output they produce for ERM
3. Retrain using "Group-Balancing" to ensure in each batch each group appears equally.



In [1]:
!pip install spuco

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 24.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.4/127.4 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 37.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s 

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader


from spuco.datasets import SpuCoMNIST, SpuriousFeatureDifficulty
from spuco.robust_train import ERM, GroupBalanceBatchERM, ClassBalanceBatchERM
from spuco.models import model_factory
from spuco.utils import Trainer
from spuco.evaluate import Evaluator
from spuco.group_inference import Cluster, ClusterAlg

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



classes = [[0, 1], [2, 3], [4, 5], [6, 7], [8, 9]]


# I chose these params for SpuriousFeatureDifficulty and SpuriousCorrelationStrength
# (Default SpuriousCorrelationStrength is 0.0 per the documentation but this is invalid for a train split)
# There wasn't a default SpuriousFeatureDifficulty in the documentation so I chose MAGNITUDE_LARGE



difficulty = SpuriousFeatureDifficulty.MAGNITUDE_LARGE
spurious_correlation_strength = 0.90



train_dataset = SpuCoMNIST(
    root="data/",
    spurious_feature_difficulty=difficulty,
    spurious_correlation_strength=spurious_correlation_strength,
    classes=classes,
    split="train",
    download=True
)


test_dataset = SpuCoMNIST(
    root="data/",
    spurious_feature_difficulty=difficulty,
    classes=classes,
    split="test",
    download=True
)
train_dataset.initialize()
test_dataset.initialize()

100%|██████████| 9.91M/9.91M [00:00<00:00, 41.8MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 1.20MB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 10.6MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 8.24MB/s]


**1)** Training a LeNet using ERM

In [4]:
# FOR MY MODEL I CHOSE A LeNet

model = model_factory("lenet", train_dataset[0][0].shape, train_dataset.num_classes).to(device)

optimizer = optim.SGD(model.parameters(), lr=1e-3, weight_decay=6e-4, momentum = 0.90, nesterov=True)


# For the model, I used the Trainer class instead of the ERM class
# because Trainer's get_trainset_outputs() method is very convenient to obtain
# the model's outputs on the training dataset, which is necessary for the next step (clustering)

# I did not see an equivalent method for the ERM class on its SpuCo documentation page
# I also did not find a way to access ERM's model member, which would have been helpful to create a helper
# function that has the same functionality as get_trainset_outputs()



erm = Trainer(
    trainset=train_dataset,
    model=model,
    batch_size=64,
    optimizer=optimizer,
    device=device,
    verbose=True
)

erm.train(num_epochs=2)

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
Epoch 0:   1%|          | 6/751 [00:00<01:20,  9.28batch/s, accuracy=17.1875%, loss=1.61]

 | Epoch 0 | Loss: 1.6078457832336426 | Accuracy: 25.0%
 | Epoch 0 | Loss: 1.6167865991592407 | Accuracy: 18.75%
 | Epoch 0 | Loss: 1.6229197978973389 | Accuracy: 12.5%
 | Epoch 0 | Loss: 1.619679570198059 | Accuracy: 15.625%
 | Epoch 0 | Loss: 1.6206260919570923 | Accuracy: 15.625%
 | Epoch 0 | Loss: 1.605092167854309 | Accuracy: 26.5625%
 | Epoch 0 | Loss: 1.6084847450256348 | Accuracy: 23.4375%
 | Epoch 0 | Loss: 1.6137419939041138 | Accuracy: 21.875%
 | Epoch 0 | Loss: 1.6144967079162598 | Accuracy: 17.1875%


Epoch 0:   2%|▏         | 16/751 [00:01<00:32, 22.74batch/s, accuracy=26.5625%, loss=1.6] 

 | Epoch 0 | Loss: 1.6124110221862793 | Accuracy: 21.875%
 | Epoch 0 | Loss: 1.607881784439087 | Accuracy: 23.4375%
 | Epoch 0 | Loss: 1.6159592866897583 | Accuracy: 18.75%
 | Epoch 0 | Loss: 1.6198073625564575 | Accuracy: 17.1875%
 | Epoch 0 | Loss: 1.6104379892349243 | Accuracy: 21.875%
 | Epoch 0 | Loss: 1.5959687232971191 | Accuracy: 32.8125%
 | Epoch 0 | Loss: 1.6046741008758545 | Accuracy: 25.0%
 | Epoch 0 | Loss: 1.619891881942749 | Accuracy: 15.625%
 | Epoch 0 | Loss: 1.6172528266906738 | Accuracy: 17.1875%
 | Epoch 0 | Loss: 1.6025885343551636 | Accuracy: 26.5625%


Epoch 0:   3%|▎         | 26/751 [00:01<00:21, 33.00batch/s, accuracy=15.625%, loss=1.62]

...

Epoch 1:  99%|█████████▉| 747/751 [00:19<00:00, 42.91batch/s, accuracy=92.1875%, loss=0.357]

 | Epoch 1 | Loss: 0.611571192741394 | Accuracy: 85.9375%
 | Epoch 1 | Loss: 0.3768048882484436 | Accuracy: 92.1875%
 | Epoch 1 | Loss: 0.32959070801734924 | Accuracy: 93.75%
 | Epoch 1 | Loss: 0.474111944437027 | Accuracy: 89.0625%
 | Epoch 1 | Loss: 0.2778753340244293 | Accuracy: 95.3125%
 | Epoch 1 | Loss: 0.5926662087440491 | Accuracy: 85.9375%
 | Epoch 1 | Loss: 0.4962042570114136 | Accuracy: 89.0625%
 | Epoch 1 | Loss: 0.27330970764160156 | Accuracy: 95.3125%
 | Epoch 1 | Loss: 0.277408242225647 | Accuracy: 95.3125%
 | Epoch 1 | Loss: 0.35667112469673157 | Accuracy: 92.1875%


Epoch 1:  99%|█████████▉| 747/751 [00:19<00:00, 42.91batch/s, accuracy=100.0%, loss=0.0982]

 | Epoch 1 | Loss: 0.5328569412231445 | Accuracy: 87.5%
 | Epoch 1 | Loss: 0.3122735321521759 | Accuracy: 93.75%
 | Epoch 1 | Loss: 0.3108746409416199 | Accuracy: 93.75%
 | Epoch 1 | Loss: 0.09820035845041275 | Accuracy: 100.0%


Epoch 1: 100%|██████████| 751/751 [00:19<00:00, 37.88batch/s, accuracy=100.0%, loss=0.0982]


In [5]:
evaluator = Evaluator(
    testset=test_dataset,
    group_partition=test_dataset.group_partition,
    group_weights=train_dataset.group_weights,
    batch_size=64,
    model=model,
    device=device,
    verbose=True
)

evaluator.evaluate()

Evaluating group-wise accuracy:   4%|▍         | 1/25 [00:00<00:09,  2.47it/s]

Group (0, 0) Accuracy: 100.0


Evaluating group-wise accuracy:   8%|▊         | 2/25 [00:00<00:09,  2.44it/s]

Group (0, 1) Accuracy: 0.0


Evaluating group-wise accuracy:  12%|█▏        | 3/25 [00:01<00:08,  2.46it/s]

...

Evaluating group-wise accuracy:  96%|█████████▌| 24/25 [00:10<00:00,  1.97it/s]

Group (4, 3) Accuracy: 0.0


Evaluating group-wise accuracy: 100%|██████████| 25/25 [00:11<00:00,  2.22it/s]

Group (4, 4) Accuracy: 100.0


{(0, 0): 100.0,
 (0, 1): 0.0,
 (0, 2): 0.0,
 (0, 3): 0.0,
 (0, 4): 0.0,
 (1, 0): 0.0,
 (1, 1): 100.0,
 (1, 2): 0.0,
 (1, 3): 0.0,
 (1, 4): 0.0,
 (2, 0): 0.0,
 (2, 1): 0.0,
 (2, 2): 100.0,
 (2, 3): 0.0,
 (2, 4): 0.0,
 (3, 0): 0.0,
 (3, 1): 0.0,
 (3, 2): 0.0,
 (3, 3): 100.0,
 (3, 4): 0.0,
 (4, 0): 0.0,
 (4, 1): 0.0,
 (4, 2): 0.0,
 (4, 3): 0.0,
 (4, 4): 100.0}

**2)** Cluster inputs based on ERM outputs

In [6]:
cluster = Cluster(
    Z = erm.get_trainset_outputs(),
    class_labels=train_dataset.labels,
    cluster_alg=ClusterAlg.KMEANS,
    num_clusters=2,
    device=device,
    verbose=True


)




group_partition = cluster.infer_groups()


keys = sorted(group_partition.keys())

for key in keys:
    print(key, len(group_partition[key]))

Clustering class-wise: 100%|██████████| 5/5 [00:00<00:00, 15.62it/s]

(0, 0) 9119
(0, 1) 1014
(1, 0) 8704
(1, 1) 968
(2, 0) 8109
(2, 1) 902
(3, 0) 975
(3, 1) 8772
(4, 0) 8496
(4, 1) 945


In [7]:
evaluator = Evaluator(
    testset=train_dataset,
    group_partition=group_partition,
    group_weights=train_dataset.group_weights,
    batch_size=64,
    model=model,
    device=device,
    verbose=True
)
evaluator.evaluate()

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
Evaluating group-wise accuracy:  10%|█         | 1/10 [00:01<00:17,  1.93s/it]

Group (0, 0) Accuracy: 100.0


Evaluating group-wise accuracy:  20%|██        | 2/10 [00:02<00:08,  1.11s/it]

Group (0, 1) Accuracy: 0.0


Evaluating group-wise accuracy:  30%|███       | 3/10 [00:04<00:10,  1.50s/it]

...

Evaluating group-wise accuracy:  90%|█████████ | 9/10 [00:12<00:01,  1.61s/it]

Group (4, 0) Accuracy: 100.0


Evaluating group-wise accuracy: 100%|██████████| 10/10 [00:12<00:00,  1.29s/it]

Group (4, 1) Accuracy: 0.0


{(0, 0): 100.0,
 (0, 1): 0.0,
 (1, 0): 100.0,
 (1, 1): 0.0,
 (2, 0): 100.0,
 (2, 1): 0.0,
 (3, 0): 0.0,
 (3, 1): 100.0,
 (4, 0): 100.0,
 (4, 1): 0.0}

**3)** Retrain using GroupBalancing

In [8]:
group_balance_erm = GroupBalanceBatchERM(
    model=model,
    num_epochs=5,
    trainset=train_dataset,
    group_partition=group_partition,
    batch_size=64,
    optimizer=optimizer,
    device=device,
    verbose=True
)
group_balance_erm.train()

Epoch 0:   1%|          | 5/751 [00:00<00:54, 13.66batch/s, accuracy=50.0%, loss=1.48]   

GB | Epoch 0 | Loss: 1.8865783214569092 | Accuracy: 48.4375%
GB | Epoch 0 | Loss: 1.5154821872711182 | Accuracy: 57.8125%
GB | Epoch 0 | Loss: 1.848420262336731 | Accuracy: 46.875%
GB | Epoch 0 | Loss: 1.7309883832931519 | Accuracy: 48.4375%
GB | Epoch 0 | Loss: 1.9460362195968628 | Accuracy: 40.625%
GB | Epoch 0 | Loss: 1.5894079208374023 | Accuracy: 53.125%
GB | Epoch 0 | Loss: 1.4343130588531494 | Accuracy: 48.4375%
GB | Epoch 0 | Loss: 1.4849287271499634 | Accuracy: 50.0%


Epoch 0:   2%|▏         | 14/751 [00:00<00:26, 27.78batch/s, accuracy=39.0625%, loss=1.58]

GB | Epoch 0 | Loss: 1.8947514295578003 | Accuracy: 35.9375%
GB | Epoch 0 | Loss: 1.180273413658142 | Accuracy: 59.375%
GB | Epoch 0 | Loss: 1.4245887994766235 | Accuracy: 45.3125%
GB | Epoch 0 | Loss: 1.3815854787826538 | Accuracy: 51.5625%
GB | Epoch 0 | Loss: 1.3836015462875366 | Accuracy: 46.875%
GB | Epoch 0 | Loss: 1.5334645509719849 | Accuracy: 48.4375%
GB | Epoch 0 | Loss: 1.3834136724472046 | Accuracy: 46.875%
GB | Epoch 0 | Loss: 1.5821274518966675 | Accuracy: 39.0625%


Epoch 0:   3%|▎         | 23/751 [00:00<00:20, 34.98batch/s, accuracy=62.5%, loss=1.21]   

...

Epoch 4:  99%|█████████▉| 747/751 [00:23<00:00, 35.26batch/s, accuracy=93.75%, loss=0.179]

GB | Epoch 4 | Loss: 0.07891733199357986 | Accuracy: 96.875%
GB | Epoch 4 | Loss: 0.1466938853263855 | Accuracy: 95.3125%
GB | Epoch 4 | Loss: 0.21163320541381836 | Accuracy: 90.625%
GB | Epoch 4 | Loss: 0.4381946921348572 | Accuracy: 90.625%
GB | Epoch 4 | Loss: 0.2511385679244995 | Accuracy: 92.1875%
GB | Epoch 4 | Loss: 0.19119538366794586 | Accuracy: 92.1875%
GB | Epoch 4 | Loss: 0.2853103280067444 | Accuracy: 90.625%
GB | Epoch 4 | Loss: 0.17905418574810028 | Accuracy: 93.75%


Epoch 4: 100%|██████████| 751/751 [00:23<00:00, 36.42batch/s, accuracy=100.0%, loss=0.00991]

GB | Epoch 4 | Loss: 0.1614154577255249 | Accuracy: 93.75%
GB | Epoch 4 | Loss: 0.15392056107521057 | Accuracy: 96.875%
GB | Epoch 4 | Loss: 0.10824679583311081 | Accuracy: 96.875%
GB | Epoch 4 | Loss: 0.009913666173815727 | Accuracy: 100.0%


Epoch 4: 100%|██████████| 751/751 [00:23<00:00, 31.92batch/s, accuracy=100.0%, loss=0.00991]


In [10]:
evaluator = Evaluator(
    testset=test_dataset,
    group_partition=test_dataset.group_partition,
    group_weights=train_dataset.group_weights,
    batch_size=64,
    model=model,
    device=device,
    verbose=True
)
evaluator.evaluate()

Evaluating group-wise accuracy:   4%|▍         | 1/25 [00:00<00:10,  2.30it/s]

Group (0, 0) Accuracy: 98.58156028368795


Evaluating group-wise accuracy:   8%|▊         | 2/25 [00:00<00:10,  2.29it/s]

Group (0, 1) Accuracy: 93.3806146572104


Evaluating group-wise accuracy:  12%|█▏        | 3/25 [00:01<00:09,  2.30it/s]

...

Evaluating group-wise accuracy:  96%|█████████▌| 24/25 [00:11<00:00,  1.73it/s]

Group (4, 3) Accuracy: 85.35353535353535


Evaluating group-wise accuracy: 100%|██████████| 25/25 [00:11<00:00,  2.13it/s]

Group (4, 4) Accuracy: 94.94949494949495


{(0, 0): 98.58156028368795,
 (0, 1): 93.3806146572104,
 (0, 2): 95.98108747044917,
 (0, 3): 96.45390070921985,
 (0, 4): 96.6903073286052,
 (1, 0): 94.37652811735941,
 (1, 1): 96.33251833740832,
 (1, 2): 92.40196078431373,
 (1, 3): 90.93137254901961,
 (1, 4): 86.27450980392157,
 (2, 0): 86.66666666666667,
 (2, 1): 87.73333333333333,
 (2, 2): 95.46666666666667,
 (2, 3): 87.2,
 (2, 4): 77.54010695187166,
 (3, 0): 91.4572864321608,
 (3, 1): 89.16876574307305,
 (3, 2): 90.93198992443325,
 (3, 3): 95.71788413098237,
 (3, 4): 91.68765743073048,
 (4, 0): 89.92443324937028,
 (4, 1): 76.57430730478589,
 (4, 2): 83.37531486146096,
 (4, 3): 85.35353535353535,
 (4, 4): 94.94949494949495}